# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
import json

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [2]:
url = 'https://scanner.tradingview.com/america/scan'
payload = {"filter":[{"left":"type","operation":"in_range","right":["stock","dr","fund"]},{"left":"subtype","operation":"in_range","right":["common","foreign-issuer","","etf","etf,odd","etf,otc","etf,cfd"]},{"left":"exchange","operation":"in_range","right":["AMEX","NASDAQ","NYSE"]},{"left":"is_primary","operation":"equal","right":True},{"left":"active_symbol","operation":"equal","right":True}],"options":{"lang":"en"},"markets":["america"],"symbols":{"query":{"types":[]},"tickers":[]},"columns":["logoid","name","close","change|1","change|5","change|15","change|60","change|240","change","change|1W","change|1M","Perf.3M","Volatility.W","average_volume_90d_calc","description","type","subtype","update_mode","pricescale","minmov","fractional","minmove2","currency","fundamental_currency_code"],"sort":{"sortBy":"name","sortOrder":"asc"},"range":[0,8033]}
data = requests.post(url, json = payload)

In [3]:
stocks_data = data.json()['data']
len(stocks_data)

8004

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
stocks_data[0]

{'s': 'NYSE:A',
 'd': ['agilent-technologies',
  'A',
  126.66,
  -0.11041009,
  -0.16552376,
  -0.18125936,
  -0.29519424,
  -1.28594809,
  -1.40888923,
  -0.75223319,
  5.33056133,
  -5.08804796,
  3.41681248,
  1940225.93333333,
  'Agilent Technologies, Inc.',
  'stock',
  'common',
  'delayed_streaming_900',
  100,
  1,
  'false',
  0,
  'USD',
  'USD']}

In [5]:
stats_from_tradingview = []

for data in stocks_data:
    stock = {}
    stock['Symbol'] = data['d'][1]
    stock['Name'] = data['d'][14]
    stock['Price'] = data['d'][2]
    stock['1min CHG'] = data['d'][3]
    stock['5min CHG'] = data['d'][4]
    stock['15min CHG'] = data['d'][5]
    stock['1Hr CHG'] = data['d'][6]
    stock['4Hr CHG'] = data['d'][7]
    stock['1Day CHG'] = data['d'][8]
    stock['1Week CHG'] = data['d'][9]
    stock['1Month CHG'] = data['d'][10]
    stock['3Month CHG'] = data['d'][11]
    stock['Volatility'] = data['d'][12]
    stock['AvgVolume'] = data['d'][13]
    
    stats_from_tradingview.append(stock)

In [6]:
stats_from_tradingview[0]['5min CHG']

-0.16552376

In [7]:
# Serializing json
json_object = json.dumps(stats_from_tradingview)
 
# Writing to sample.json
with open("stocks_all_performance_short_term.json", "w") as outfile:
    outfile.write(json_object)

In [8]:
hqm_dataframe = pd.read_json('stocks_all_performance_short_term.json')
available_stocks = pd.read_json('curated_stocks_all.json')
available_stocks_list = available_stocks['symbol'].tolist()

In [9]:
hqm_dataframe = hqm_dataframe[hqm_dataframe['Symbol'].isin(available_stocks_list)]

In [10]:
hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,1Month CHG,3Month CHG,Volatility,AvgVolume
0,A,"Agilent Technologies, Inc.",126.6600,-0.110410,-0.165524,-0.181259,-0.295194,-1.285948,-1.408889,-0.752233,5.330561,-5.088048,3.416812,1.940226e+06
1,AA,Alcoa Corporation,33.9100,-0.176626,-0.097222,-0.235363,-0.455012,-2.093258,-2.669346,1.587777,-0.058945,-6.014412,3.848276,4.831685e+06
11,AAMC,Altisource Asset Management Corp Com,37.8500,0.105792,0.705069,1.068091,0.987193,1.610738,-5.845771,-22.565466,-32.639260,-55.885781,8.061637,2.458362e+04
12,AAME,Atlantic American Corporation,1.8700,-2.094241,-2.094241,-2.094241,-2.094241,-1.058201,-3.608247,1.081081,-3.108808,-14.224118,3.402829,5.556644e+03
13,AAN,"Aarons Holdings Company, Inc.",15.3800,-0.065020,0.457516,0.391901,0.367645,-1.474359,-1.975781,-1.787995,8.769448,15.205993,3.399962,3.339910e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,ZUO,"Zuora, Inc.",10.5100,-0.237417,0.000000,0.047619,0.190749,-0.989632,-0.849057,0.000000,-4.193254,34.056122,3.231262,9.495181e+05
7997,ZVIA,Zevia PBC,2.7200,-0.367647,-0.367647,-1.411525,-2.166065,-5.847201,-10.819672,-35.083532,-36.890951,-18.318318,9.174806,1.489865e+05
8001,ZYME,Zymeworks Inc.,7.2800,-0.068729,0.345066,0.137741,-0.615174,-0.886162,-3.062583,-7.496823,-15.740741,-13.744076,4.713737,6.186621e+05
8002,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.3551,0.028169,0.028169,-0.112518,-0.308815,-1.906077,-2.445055,-1.361111,8.926380,-9.873096,4.966745,1.942771e+05


In [11]:
mean_val = hqm_dataframe['AvgVolume'].mean()
hqm_dataframe['AvgVolume'].fillna(value=mean_val, inplace=True)
mean_val = hqm_dataframe['1Week CHG'].mean()
hqm_dataframe['1Week CHG'].fillna(value=mean_val, inplace=True)
hqm_dataframe.isna().any()

Symbol        False
Name          False
Price         False
1min CHG      False
5min CHG      False
15min CHG     False
1Hr CHG       False
4Hr CHG       False
1Day CHG      False
1Week CHG     False
1Month CHG    False
3Month CHG    False
Volatility    False
AvgVolume     False
dtype: bool

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Cleaning Data
Clean all rows that have no historical Price Return, as in Nonetype

In [12]:
# for row in hqm_dataframe.index:
#     if(hqm_dataframe.loc[row, '1yearPerformace'] == None):
#         hqm_dataframe.drop(row, inplace=True)

hqm_dataframe.reset_index(inplace=True, drop=True)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [13]:
hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,1Month CHG,3Month CHG,Volatility,AvgVolume
0,A,"Agilent Technologies, Inc.",126.6600,-0.110410,-0.165524,-0.181259,-0.295194,-1.285948,-1.408889,-0.752233,5.330561,-5.088048,3.416812,1.940226e+06
1,AA,Alcoa Corporation,33.9100,-0.176626,-0.097222,-0.235363,-0.455012,-2.093258,-2.669346,1.587777,-0.058945,-6.014412,3.848276,4.831685e+06
2,AAMC,Altisource Asset Management Corp Com,37.8500,0.105792,0.705069,1.068091,0.987193,1.610738,-5.845771,-22.565466,-32.639260,-55.885781,8.061637,2.458362e+04
3,AAME,Atlantic American Corporation,1.8700,-2.094241,-2.094241,-2.094241,-2.094241,-1.058201,-3.608247,1.081081,-3.108808,-14.224118,3.402829,5.556644e+03
4,AAN,"Aarons Holdings Company, Inc.",15.3800,-0.065020,0.457516,0.391901,0.367645,-1.474359,-1.975781,-1.787995,8.769448,15.205993,3.399962,3.339910e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3084,ZUO,"Zuora, Inc.",10.5100,-0.237417,0.000000,0.047619,0.190749,-0.989632,-0.849057,0.000000,-4.193254,34.056122,3.231262,9.495181e+05
3085,ZVIA,Zevia PBC,2.7200,-0.367647,-0.367647,-1.411525,-2.166065,-5.847201,-10.819672,-35.083532,-36.890951,-18.318318,9.174806,1.489865e+05
3086,ZYME,Zymeworks Inc.,7.2800,-0.068729,0.345066,0.137741,-0.615174,-0.886162,-3.062583,-7.496823,-15.740741,-13.744076,4.713737,6.186621e+05
3087,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.3551,0.028169,0.028169,-0.112518,-0.308815,-1.906077,-2.445055,-1.361111,8.926380,-9.873096,4.966745,1.942771e+05


In [14]:
list(hqm_dataframe.columns.values)

['Symbol',
 'Name',
 'Price',
 '1min CHG',
 '5min CHG',
 '15min CHG',
 '1Hr CHG',
 '4Hr CHG',
 '1Day CHG',
 '1Week CHG',
 '1Month CHG',
 '3Month CHG',
 'Volatility',
 'AvgVolume']

In [15]:
time_periods = [
    '1min CHG',
    '5min CHG',
    '15min CHG',
    '1Hr CHG',
    '4Hr CHG',
    '1Day CHG',
    '1Week CHG',
#     '1Month CHG',
#     '3Month CHG',
    'Volatility'
]


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period}'
        percentile_col = f'{time_period} Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,...,Volatility,AvgVolume,1min CHG Percentile,5min CHG Percentile,15min CHG Percentile,1Hr CHG Percentile,4Hr CHG Percentile,1Day CHG Percentile,1Week CHG Percentile,Volatility Percentile
0,A,"Agilent Technologies, Inc.",126.6600,-0.110410,-0.165524,-0.181259,-0.295194,-1.285948,-1.408889,-0.752233,...,3.416812,1.940226e+06,24.700550,17.222402,21.689867,26.286824,34.347685,44.318550,46.973130,52.929751
1,AA,Alcoa Corporation,33.9100,-0.176626,-0.097222,-0.235363,-0.455012,-2.093258,-2.669346,1.587777,...,3.848276,4.831685e+06,17.513758,22.434445,19.261897,18.679184,15.765620,23.146649,71.026222,61.022985
2,AAMC,Altisource Asset Management Corp Com,37.8500,0.105792,0.705069,1.068091,0.987193,1.610738,-5.845771,-22.565466,...,8.061637,2.458362e+04,74.263516,91.421172,94.011007,92.975073,96.536096,6.733571,1.068307,90.644221
3,AAME,Atlantic American Corporation,1.8700,-2.094241,-2.094241,-2.094241,-2.094241,-1.058201,-3.608247,1.081081,...,3.402829,5.556644e+03,1.392036,1.424409,1.780511,2.622208,40.660408,15.150534,66.623503,52.670767
4,AAN,"Aarons Holdings Company, Inc.",15.3800,-0.065020,0.457516,0.391901,0.367645,-1.474359,-1.975781,-1.787995,...,3.399962,3.339910e+05,33.085141,84.363872,72.450631,76.400129,28.617676,33.635481,36.225316,52.573648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3084,ZUO,"Zuora, Inc.",10.5100,-0.237417,0.000000,0.047619,0.190749,-0.989632,-0.849057,0.000000,...,3.231262,9.495181e+05,13.240531,36.176756,41.372612,66.558757,43.185497,57.656199,55.454840,49.368728
3085,ZVIA,Zevia PBC,2.7200,-0.367647,-0.367647,-1.411525,-2.166065,-5.847201,-10.819672,-35.083532,...,9.174806,1.489865e+05,9.161541,10.359340,3.949498,2.525089,0.841696,1.359663,0.226611,93.266429
3086,ZYME,Zymeworks Inc.,7.2800,-0.068729,0.345066,0.137741,-0.615174,-0.886162,-3.062583,-7.496823,...,4.713737,6.186621e+05,32.340563,77.468436,50.242797,13.790871,46.487536,19.391389,9.970864,71.608935
3087,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.3551,0.028169,0.028169,-0.112518,-0.308815,-1.906077,-2.445055,-1.361111,...,4.966745,1.942771e+05,61.152477,43.088378,25.606993,25.250890,19.197151,25.995468,40.466170,74.004532


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [16]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)   

In [17]:
hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,...,AvgVolume,1min CHG Percentile,5min CHG Percentile,15min CHG Percentile,1Hr CHG Percentile,4Hr CHG Percentile,1Day CHG Percentile,1Week CHG Percentile,Volatility Percentile,HQM Score
0,A,"Agilent Technologies, Inc.",126.6600,-0.110410,-0.165524,-0.181259,-0.295194,-1.285948,-1.408889,-0.752233,...,1.940226e+06,24.700550,17.222402,21.689867,26.286824,34.347685,44.318550,46.973130,52.929751,33.558595
1,AA,Alcoa Corporation,33.9100,-0.176626,-0.097222,-0.235363,-0.455012,-2.093258,-2.669346,1.587777,...,4.831685e+06,17.513758,22.434445,19.261897,18.679184,15.765620,23.146649,71.026222,61.022985,31.106345
2,AAMC,Altisource Asset Management Corp Com,37.8500,0.105792,0.705069,1.068091,0.987193,1.610738,-5.845771,-22.565466,...,2.458362e+04,74.263516,91.421172,94.011007,92.975073,96.536096,6.733571,1.068307,90.644221,68.456620
3,AAME,Atlantic American Corporation,1.8700,-2.094241,-2.094241,-2.094241,-2.094241,-1.058201,-3.608247,1.081081,...,5.556644e+03,1.392036,1.424409,1.780511,2.622208,40.660408,15.150534,66.623503,52.670767,22.790547
4,AAN,"Aarons Holdings Company, Inc.",15.3800,-0.065020,0.457516,0.391901,0.367645,-1.474359,-1.975781,-1.787995,...,3.339910e+05,33.085141,84.363872,72.450631,76.400129,28.617676,33.635481,36.225316,52.573648,52.168987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3084,ZUO,"Zuora, Inc.",10.5100,-0.237417,0.000000,0.047619,0.190749,-0.989632,-0.849057,0.000000,...,9.495181e+05,13.240531,36.176756,41.372612,66.558757,43.185497,57.656199,55.454840,49.368728,45.376740
3085,ZVIA,Zevia PBC,2.7200,-0.367647,-0.367647,-1.411525,-2.166065,-5.847201,-10.819672,-35.083532,...,1.489865e+05,9.161541,10.359340,3.949498,2.525089,0.841696,1.359663,0.226611,93.266429,15.211233
3086,ZYME,Zymeworks Inc.,7.2800,-0.068729,0.345066,0.137741,-0.615174,-0.886162,-3.062583,-7.496823,...,6.186621e+05,32.340563,77.468436,50.242797,13.790871,46.487536,19.391389,9.970864,71.608935,40.162674
3087,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.3551,0.028169,0.028169,-0.112518,-0.308815,-1.906077,-2.445055,-1.361111,...,1.942771e+05,61.152477,43.088378,25.606993,25.250890,19.197151,25.995468,40.466170,74.004532,39.345257


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [18]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe.reset_index(inplace=True, drop=True)
# hqm_dataframe[hqm_dataframe['Price'] < 10][:50]

In [19]:
cols = list(hqm_dataframe.columns.values)
cols

['Symbol',
 'Name',
 'Price',
 '1min CHG',
 '5min CHG',
 '15min CHG',
 '1Hr CHG',
 '4Hr CHG',
 '1Day CHG',
 '1Week CHG',
 '1Month CHG',
 '3Month CHG',
 'Volatility',
 'AvgVolume',
 '1min CHG Percentile',
 '5min CHG Percentile',
 '15min CHG Percentile',
 '1Hr CHG Percentile',
 '4Hr CHG Percentile',
 '1Day CHG Percentile',
 '1Week CHG Percentile',
 'Volatility Percentile',
 'HQM Score']

In [20]:
# rearrange columns
cols = list(hqm_dataframe.columns.values)
hqm_dataframe = hqm_dataframe[['Symbol',
 'Name',
 'Price',
 'HQM Score',
 'AvgVolume',
 'Volatility',
 '1min CHG',
 '5min CHG',
 '15min CHG',
 '1Hr CHG',
 '4Hr CHG',
 '1Day CHG',
 '1Week CHG',
#  '1Month CHG',
#  '3Month CHG',
 '1min CHG Percentile',
 '5min CHG Percentile',
 '15min CHG Percentile',
 '1Hr CHG Percentile',
 '4Hr CHG Percentile',
 '1Day CHG Percentile',
 '1Week CHG Percentile',
#  '1Month CHG Percentile',
#  '3Month CHG Percentile',
 'Volatility Percentile'
]]

In [21]:
hqm_dataframe[:50]

,Symbol,Name,Price,HQM Score,AvgVolume,Volatility,1min CHG,5min CHG,15min CHG,1Hr CHG,...,1Day CHG,1Week CHG,1min CHG Percentile,5min CHG Percentile,15min CHG Percentile,1Hr CHG Percentile,4Hr CHG Percentile,1Day CHG Percentile,1Week CHG Percentile,Volatility Percentile
0,ASTI,"Ascent Solar Technologies, Inc",0.088000,98.539171,1.635227e+06,9.377927,6.329114,6.329114,4.477612,4.477612,...,11.392405,11.392405,99.870508,99.870508,99.514406,99.417287,99.579152,99.352541,97.086436,93.622532
1,VALU,"Value Line, Inc.",57.500000,96.969084,2.407989e+03,9.182290,2.192426,2.192426,2.192426,2.192426,...,12.370530,5.504587,98.899320,98.705083,98.251861,97.928132,98.899320,99.417287,90.352865,93.298802
2,AKTX,Akari Therapeutics Plc,0.212000,96.103108,5.842267e+05,12.766964,3.163017,0.355030,3.163017,4.228122,...,7.942974,21.839080,99.384914,78.245387,99.093558,99.223050,97.021690,98.769828,99.384914,97.701522
3,MIND,"MIND Technology, Inc.",0.719000,95.726772,1.454677e+05,9.573723,3.956534,3.956534,3.956534,3.956534,...,2.714286,11.128284,99.482033,99.482033,99.352541,99.028812,83.166073,94.496601,96.924571,93.881515
4,AWX,Avalon Holdings Corporation,2.600000,94.642279,7.124000e+03,4.219463,8.333333,8.333333,8.333333,8.333333,...,8.333333,6.557377,99.967627,99.967627,99.935254,99.935254,99.805762,98.834574,92.683716,66.008417
5,AUUD,Auddia Inc.,0.432000,94.626093,2.981482e+06,7.899054,2.357143,5.964999,2.381763,2.601432,...,0.232019,13.177888,98.996439,99.805762,98.478472,98.478472,94.108126,79.281321,97.992878,89.867271
6,ICCC,ImmuCell Corporation,5.510000,94.589673,9.888756e+03,7.611738,3.962264,3.962264,3.962264,3.962264,...,2.416357,2.416357,99.514406,99.514406,99.384914,99.061185,99.190677,93.849142,77.500809,88.701845
7,LIQT,"LiqTech International, Inc.",3.650000,94.460181,1.323279e+04,4.335407,2.097902,2.097902,2.097902,2.097902,...,7.688676,11.280488,98.737455,98.543218,98.057624,97.733894,99.384914,98.737455,96.989317,67.497572
8,FLUX,"Flux Power Holdings, Inc.",5.560000,94.180965,5.357744e+04,7.868737,0.361011,0.907441,1.632332,2.394107,...,10.978044,11.646586,89.122693,94.237617,96.956944,98.251861,88.669472,99.287795,97.215927,89.705406
9,LUMO,"Lumos Pharma, Inc.",3.340000,94.164778,1.955707e+04,7.066059,1.212121,2.140673,2.453988,1.519757,...,2.769231,3.726708,96.892198,98.672710,98.575591,96.503723,97.636776,94.626093,84.040142,86.370994
